In [1]:
import numpy
import math

from quantization import Tensor
from quantization_utils import *
from data_utils import *

# Fixed Point

---
## What's Fixed Point?

You may refer [here](https://stackoverflow.com/questions/7524838/fixed-point-vs-floating-point-number) for the difference between fixed point & floating point.

We usually use [Qm.n notation](https://en.wikipedia.org/wiki/Q_%28number_format%29) to represent the fixed point.

For example Q3.12 means we use 3 bits to represent the integer parts and 12 bits to represent the fractional parts (and one bit to sign).

So the range for Qm.n is [-2^m, 2^(m+1) - 2^-n]

---
## Fixed Point <-> Floating point?

In this case, we will represent all the fixed point using integer values (int16, int32, etc.), and the floating value is simply:

$$value_{float} = value_{int} * 2^{-frac_{bits}}$$

where $$frac_{bits} = total_{bits} - integer_{bits} - 1$$

So convert floating point value x to Qm.n format is simply:

$$value_{int} = value_{float} * 2^{n}$$

---
## Why Fixed Point matters?

Fixed point arithmetics can be very useful for division, tanh, exp and others.

This codelab will focus on division mainly with brief introduction to others.

# Fixed Point Arithmetic

Let's just focus on Qm.n case first. It's easier to convert Qm.n into Qm1.n1 case (where just bits shifting).

---
## Add

So we have

$$lhs_{float} = lhs_{int} * 2^{-n}$$

and

$$rhs_{float} = rhs_{int} * 2^{-n}$$

Our desired output is:

$$out_{int} = (lhs_{float} + rhs_{float}) * 2^{n}$$

=>

$$out_{int} = (lhs_{int} * 2^{-n} + rhs_{int} * 2^{-n}) * 2^{n}$$

=>

$$out_{int} = (lhs_{int} + rhs_{int}) * 2^{-n} * 2^{n}$$

=>

$$out_{int} = lhs_{int} + rhs_{int}$$

So, the output is simply the add of the int value of lhs and rhs, exactly like the int math! Very simple, right?

---
## Sub

Sub is actually very similar to add, so we can just skip the deduction part :)

We will have

$$out_{int} = lhs_{int} - rhs_{int}$$

for the sub.

---
## Mul

Let's take a look at the mul:

Still we have:

$$lhs_{float} = lhs_{int} * 2^{-n}$$

and

$$rhs_{float} = rhs_{int} * 2^{-n}$$

Our desired output is:

$$out_{int} = (lhs_{float} * rhs_{float}) * 2^{n}$$

=>

$$out_{int} = (lhs_{int} * 2^{-n} * rhs_{int} * 2^{-n}) * 2^{n}$$

=>

$$out_{int} = (lhs_{int} * rhs_{int}) * 2^{-n} * 2^{-n} * 2^{n}$$

=>

$$out_{int} = (lhs_{int} * rhs_{int}) * 2^{-n}$$

Given the reality that n is always >= 0, so we essentially have a normal integer mul with a right shift of n bits.

## Div

---

Div is indeed very complicated as we're limited by two constraints: 1) we're dealing with integer-only arithmetic; 2) most of the time, we don't have direct and accurate 'div' instruction available, so we really need to emulate the 'div' behavior.

How are we going to do that exactly?

---

### Newton-Ralpson Division

See the detailed reference [here](https://en.wikipedia.org/wiki/Division_algorithm#Newton.E2.80.93Raphson_division)

The first observation is we can transform:

$$out = lhs \div rhs$$

into $$out = lhs * (\frac{1}{rhs})$$

and for x in range of [0.5, 1], we can use the newton-ralpson method to estimate the value.

Initial value is:

$$x_0 = (\frac{48}{17}) - (\frac{32}{17}) * x$$

Then we can iterate using:

$$x_{new} = x_{pre} + x_{pre} * (1 - x * x_{pre})$$

Normally we can get good accuracy with 2 or 3 iterations, let's just verify it:

In [2]:
def newton_ralpson_reci(x, iteration=2):
  a = 48.0 / 17.0
  b = 32.0 / 17.0
  result = a - b * x
  for i in range(iteration):
    result = result + result * (1 - x * result)
  return result

test_data = np.arange(0.5, 1, 0.001)

expected_result = 1.0 / test_data

result_iter2 = newton_ralpson_reci(test_data, 2)
result_iter3 = newton_ralpson_reci(test_data, 3)

print("Avg Diff for 2 iterations is %s" % (diff(expected_result, result_iter2)))
print("Avg Diff for 3 iterations is %s" % (diff(expected_result, result_iter3)))
print("Max Diff for 2 iterations is %s" % (max_diff(expected_result, result_iter2)))
print("Max Diff for 3 iterations is %s" % (max_diff(expected_result, result_iter3)))

Avg Diff for 2 iterations is 5.553048509880476e-06
Avg Diff for 3 iterations is 4.685110077673471e-11
Max Diff for 2 iterations is 2.394607344258226e-05
Max Diff for 3 iterations is 2.867073245482743e-10


### Reciprocal
---

So far so good, but wait a second, isn't newton_ralpson_reci only works for [0.5, 1]? 

Well, we can always "canonicalize" the value to make it between [0.5, 1] with a proper shift by counting the leading zeros (in arm, it's just instruction 'CLS'). And yes, that only applies to positive numbers, but it should be very cheap for negative values to apply 'NEG'.

---
#### Canonicalize

So for Q0.n+m case (all bits go to the fractional bits except the sign bits), we just to make sure the number of leading zeros is 1 (which is the sign bit).

So we have:

$$value_{FloatQ0.n+m} = value_{IntQm.n} * 2^{m} * 2^{-(m+n)}$$

It's easily to get

$$value_{IntQ0.n+m} = value_{IntQm.n} * 2^{m}$$

In reality, simply shifting Qm.n int value left by m bits can easily result in overflow, so we don't shift it just yet, but just do that at the end.

Moving forward:

We know

$$value_{IntCano} = value_{IntQ0.n+m} * 2^{zeros_{leading} - 1}$$

so we have

$$value_{FloatCano} = value_{IntQ0.n+m} * 2^{zeros_{leading} - 1} * 2^{-(zeros_{leading} - 1)} * 2^{-(m+n)}$$

=>

$$value_{FloatCano} = value_{IntQm.n} * 2^{m} * 2^{zeros_{leading} - 1} * 2^{-(zeros_{leading} - 1)} * 2^{-(m+n)}$$

=>

$$value_{FloatCano} = (value_{IntQm.n} * 2^{zeros_{leading} - 1} * 2^{-(m+n)}) * 2^{m-(zeros_{leading} - 1)}$$

Let 

$$value_{shifted} = value_{IntQm.n} * 2^{zeros_{leading} - 1} * 2^{-(m+n)}$$

So

$$\frac{1}{value_{FloatCano}} = f(value_{shifted}) * 2^{(zeros_{leading} - 1) - m}$$

and $$value_{shifted}$$ can be interpreted as the canonicalize representation for Q0.m+n format.

where function f is the newton-ralpson method.

---

Another view of this problem, when we transform Qm.n into Q0.m+n format, we need to shift left by m bits, but we don't shift that just yet, we can shift leading_zeros - 1, so we will still need to shift left by m + 1 - leading_zeros bits.

---

In reality, since for newton-ralpson method we need to represent 48/17, which will require Q2.m+n-2 representation, note, from Q0.m+n to Q2.m+n-2 is just to shift right by two bits or shift left by -2 bits.

In [3]:
def clz(value):
  assert value >= 0
  if value.dtype == np.int32:
    total_bits = 32
  elif vlaue.dtype == np.int16:
    total_bits = 16
  else:
    assert False
    
  return total_bits - (len(bin(value)) - 2)

In [4]:
class FixedPoint:
  def __init__(self, integer, fractional, datatype=np.int32, scale=None, zero_point=None):
    # datatype is either np.int16 or np.int32.
    self.integer = integer
    self.fractional = fractional
    self.datatype = datatype
    self.value = None
    self.scale = scale
    self.zero_point = zero_point
    if self.scale:
      self.multiplier = self.from_float(self.scale)
    
  def set_raw_value(self, value):
    self.value = value
    
  def get_raw_value(self):
    return self.value
    
  def to_float(self):
    return self.value * (2 ** (-self.fractional))

  def from_float(self, value):
    temp = value * (2 ** self.fractional)
    if self.datatype == np.int32:
      max_value = 2 ** 31 - 1
      min_value = - 2 ** 31
    elif self.datatype == np.int16:
      max_value = 2 ** 15 - 1
      min_value = -2 ** 15
    else:
      # Should not reached
      assert False
    
    temp = max(min(temp, max_value), min_value)
    return self.datatype(temp)
    
  def from_q_tensor(self, value):
    # This may result in overflow
    return (value - self.zero_point) * self.multiplier

  # currently we only support same type arithmetics.
  def check(self, another):
    assert self.integer == another.integer
    assert self.fractional == another.fractional
    assert self.datatype == another.datatype
    
  def add(self, another):
    self.check(another)
    result = FixedPoint(self.integer, self.fractional, self.datatype, self.scale, self.zero_point)
    # We need to be careful about overflow.
    result.set_raw_value(self.get_raw_value() + another.get_raw_value())
    return result

  def sub(self, another):
    self.check(another)
    result = FixedPoint(self.integer, self.fractional, self.datatype, self.scale, self.zero_point)
    # We need to be careful about overflow.
    result.set_raw_value(self.get_raw_value() - another.get_raw_value())
    return result

  def mul(self, another):
    self.check(another)
    result = FixedPoint(self.integer, self.fractional, self.datatype, self.scale, self.zero_point)
    # We need to be careful about overflow.
    temp = np.int64(self.get_raw_value()) * np.int64(another.get_raw_value())
    temp = rounding_divide_by_POT(temp, self.fractional)                        
    result.set_raw_value(temp)
    return result

  def neg(self):
    result = FixedPoint(self.integer, self.fractional, self.datatype, self.scale, self.zero_point)
    assert self.get_raw_value() != None
    result.set_raw_value(-self.get_raw_value())
    return result

  # this only handles cannonical format of Q2.x.
  def _reciprocal_for_cano(self, iteration=2):
    a = FixedPoint(2, 29, np.int32)
    a.set_raw_value(a.from_float(48.0 / 17.0))
    b = FixedPoint(2, 29, np.int32)
    b.set_raw_value(a.from_float(32.0 / 17.0))
    one = FixedPoint(2, 29, np.int32)
    one.set_raw_value(one.from_float(1.0))
    result = a.sub(self.mul(b))
    for i in range(iteration):
      temp = one.sub(self.mul(result))
      temp = temp.mul(result)
      result = result.add(temp)
    return result

  def reciprocal(self, iteration=2):
    # We should also check hte raw value should not be 0.
    if self.get_raw_value() < 0:
      return self.neg().reciprocal().neg()

    leading_zeros = clz(self.get_raw_value())
    # We need to shift left leading_zeros - 1 bits so the raw value is in the canonical format for Q0.n.
    # We also need to shift right by 2 bits so the raw_value is in the Q2.n case.
    shift_left_bits = leading_zeros - 1 - 2
    raw_value = self.get_raw_value() * (2 ** shift_left_bits)
    cano = FixedPoint(2, 29, self.datatype)
    cano.set_raw_value(raw_value)
    # Note the return is actually in Q2.n format.
    # we still need to shift left by leading_zeros bits -1 - m.
    cano_reci = cano._reciprocal_for_cano(iteration)
    final_shift_left_bits = leading_zeros - 1 - self.integer
    final_raw_value = cano_reci.get_raw_value()
    # Since we're using Q2.n, it's possible we run into shift left < -2 scenario,
    # in that sense, we need to shift right the value.
    if final_shift_left_bits < -2:
      value_shift_right_bits = -2 - final_shift_left_bits
      final_raw_value = rounding_divide_by_POT(final_raw_value, value_shift_right_bits)
      final_shift_left_bits = -2
    result = FixedPoint(cano_reci.integer + final_shift_left_bits,
                        cano_reci.fractional - final_shift_left_bits,
                        cano_reci.datatype)
    result.set_raw_value(final_raw_value)
    return result

In [5]:
# Let's verify reciprocal for the canonical format first.
cano_test_data = np.arange(0.5, 0.99, 0.001)

cano_expect_result = []
cano_true_result = []
for item in cano_test_data:
  expect_value = 1.0 / item
  test = FixedPoint(2, 29, np.int32)
  test.set_raw_value(test.from_float(item))
  result = test.reciprocal().to_float()
  cano_expect_result.append(expect_value)
  cano_true_result.append(result)
    
cano_expect_result = np.array(cano_expect_result)
cano_true_result = np.array(cano_true_result)

print("Avg Diff is %s" % (diff(cano_expect_result, cano_true_result)))
print("Avg Diff is %s" % (diff(cano_expect_result, cano_true_result)))
print("Max Diff is %s" % (max_diff(cano_expect_result, cano_true_result)))
print("Max Diff is %s" % (max_diff(cano_expect_result, cano_true_result)))

Avg Diff is 5.492057929116291e-06
Avg Diff is 5.492057929116291e-06
Max Diff is 2.3948337191770364e-05
Max Diff is 2.3948337191770364e-05


In [6]:
# Let's also verify reciprocal for a wider range (including negative values.)
test_data = np.arange(-10.5, 10.5, 0.2)

expect_result = []
true_result = []
for item in test_data:
  expect_value = 1.0 / item
  test = FixedPoint(4, 27, np.int32)
  test.set_raw_value(test.from_float(item))
  test = test.reciprocal()
  result = test.to_float()
  expect_result.append(expect_value)
  true_result.append(result)

expect_result = np.array(expect_result)
true_result = np.array(true_result)

print("Avg Diff is %s" % (diff(expect_result, true_result)))
print("Max Diff is %s" % (max_diff(expect_result, true_result)))

Avg Diff is 3.077765745413778e-06
Max Diff is 8.51779436708e-05


Thanks to my college tianlin, I decided to expand this codelab to cover sin. (well, if you know how to handle sin, cos should be just trivial. :) )

---
### Sin

For reference, please see details [here](https://en.wikipedia.org/wiki/Small-angle_approximation).

$$\sin \theta = \sum^{\infty}_{n=0} \frac{(-1)^n}{(2n+1)!} \theta^{2n+1}$$

And I found we can get good approximation with a few expansions (6~7), See the followig code:

In [7]:
def sin(radian, expansion=6):
  sign = 1
  divi = 1
  result = radian
  theta = radian
  for i in range(1, expansion):
    divi *= (2 * i) * (2 * i + 1)
    theta *= radian * radian
    sign *= -1
    result += sign * theta / divi
  return result

test_data = np.arange(0, math.pi, 0.01)

expect_result = np.sin(test_data)

true_result = []

for item in test_data:
  true_result.append(sin(item))

true_result = np.array(true_result)

print("Avg Diff is %s" % (diff(expect_result, true_result)))
print("Max Diff is %s" % (max_diff(expect_result, true_result)))

Avg Diff is 3.23777951746771e-05
Max Diff is 0.0004422558761439342


In [8]:
# Let's implement sin with fixed point math!

# Let's just assume radian is in Q2.29.
# Also, Let's just assume all the value are within [-pi/2, pi/2]
def sin_fixed_point(radian):
  multiplier_1 = FixedPoint(2, 29, np.int32)
  multiplier_2 = FixedPoint(2, 29, np.int32)
  multiplier_3 = FixedPoint(2, 29, np.int32)
  multiplier_4 = FixedPoint(2, 29, np.int32)
  multiplier_5 = FixedPoint(2, 29, np.int32)

  multiplier_1.set_raw_value(-89478485)
  multiplier_2.set_raw_value(4473924)
  multiplier_3.set_raw_value(-106522)
  multiplier_4.set_raw_value(1479)
  multiplier_5.set_raw_value(-13)
  
  result = radian
  radian_square = radian.mul(radian)
  
  # We're trying to avoid the overflow issue by afford some computation cost.
  multiplier_1 = multiplier_1.mul(radian_square)
  result = result.add(radian.mul(multiplier_1))
  multiplier_2 = multiplier_2.mul(radian_square).mul(radian_square)
  result = result.add(radian.mul(multiplier_2))
  multiplier_3 = multiplier_3.mul(radian_square).mul(radian_square).mul(radian_square)
  result = result.add(radian.mul(multiplier_3))
  multiplier_4 = multiplier_4.mul(radian_square).mul(radian_square).mul(radian_square).mul(radian_square)
  result = result.add(radian.mul(multiplier_4))
  multiplier_5 = multiplier_4.mul(radian_square).mul(radian_square).mul(radian_square).mul(radian_square).mul(radian_square)
  result = result.add(radian.mul(multiplier_5))
  return result

fixed_point_test_data = np.arange(-math.pi / 2.0, math.pi / 2.0, 0.01)

fixed_point_expect_result = np.sin(fixed_point_test_data)

fixed_point_true_result = []

for item in fixed_point_test_data:
  test = FixedPoint(2, 29, np.int32)
  test.set_raw_value(test.from_float(item))
  result = sin_fixed_point(test)
  fixed_point_true_result.append(result.to_float())

fixed_point_true_result = np.array(fixed_point_true_result)

print("Avg Diff is %s" % (diff(fixed_point_expect_result, fixed_point_true_result)))
print("Max Diff is %s" % (max_diff(fixed_point_expect_result, fixed_point_true_result)))

Avg Diff is 0.0007714679334449271
Max Diff is 0.014670956791459133
